# ACC calculation - 6 hourly Wxformer

* This notebook shows how ACCs were computed for 6 hourly Wxformer outputs.

* The notebook runs with `verif_config_6h.yml` in the same folder.

In [1]:
import os
import sys
import yaml
import argparse
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

### Load config

In [2]:
config_name = os.path.realpath('verif_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### Model and lead time to verify

In [3]:
model_name = 'wxformer'
lead_range = conf[model_name]['lead_range']
verif_lead_range = conf[model_name]['verif_lead_range']

leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
leads_verif = list(np.arange(verif_lead_range[0], verif_lead_range[-1]+verif_lead_range[0], verif_lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, leads_verif)

print('Verifying lead times: {}'.format(leads_verif))
print('Verifying lead indices: {}'.format(ind_lead))

Verifying lead times: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
Verifying lead indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [4]:
verif_ind_start = 0; verif_ind_end = 2*(365+366+365)

path_verif = conf[model_name]['save_loc_verif']+'combined_acc_{}_{}_{}h_{}h_{}.nc'.format(
    verif_ind_start, verif_ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)

### ACC compute

In [5]:
# ---------------------------------------------------------------------------------------- #
# temp function for lat / lon average
def sp_avg(DS, wlat):
    return DS.weighted(wlat).mean(['latitude', 'longitude'], skipna=False)

# ---------------------------------------------------------------------------------------- #
# ERA5 climatology info
ERA5_path_string = conf['ERA5_weatherbench']['save_loc_clim'] + 'ERA5_clim_1990_2019_6h_interp.nc'
ds_ERA5_clim = xr.open_dataset(ERA5_path_string)
# rename coordinate name to latitude, longitude
if 'lat' in ds_ERA5_clim.coords:
    ds_ERA5_clim = ds_ERA5_clim.rename({'lat': 'latitude'})
if 'lon' in ds_ERA5_clim.coords:
    ds_ERA5_clim = ds_ERA5_clim.rename({'lon': 'longitude'})

# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')
    
# Select the specified variables and their levels
variables_levels = conf['ERA5_ours']['verif_variables']

# subset merged ERA5
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]

L_max = len(filename_OURS)
print('Total number of initialization times: {}'.format(L_max))
assert verif_ind_end <= L_max, 'verified indices (days) exceeds the max index available'

filename_OURS = filename_OURS[verif_ind_start:verif_ind_end]

# ---------------------------------------------------------------------------------------- #
# latitude weighting
lat = xr.open_dataset(filename_OURS[0])["latitude"]
w_lat = np.cos(np.deg2rad(lat))
w_lat = w_lat / w_lat.mean()

# ---------------------------------------------------------------------------------------- #
# ACC compute

acc_results = []

for fn_ours in filename_OURS:
    
    # --------------------------------------------------------------- #
    # import and subset forecast
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variables_levels)
    ds_ours = ds_ours.isel(time=ind_lead)
    dayofyear = ds_ours['time.dayofyear']
    hourofday = ds_ours['time'].dt.hour
    ds_ours = ds_ours.load()
    
    # --------------------------------------------------------------- #
    # get ERA5 verification target
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time']).load()
    
    # --------------------------------------------------------------- #
    # get ERA5 climatology
    ds_clim_target = ds_ERA5_clim.sel(dayofyear=dayofyear, hour=hourofday).load()
    
    # ========================================== #
    # ERA5 anomaly
    ds_anomaly_ERA5 = ds_target - ds_clim_target

    # fcst anomaly
    ds_anomaly_OURS = ds_ours - ds_clim_target
    
    # ========================================== #
    # anmalies --> ACC with latitude-based cosine weights (check sp_avg and w_lat)
    top = sp_avg(ds_anomaly_OURS*ds_anomaly_ERA5, w_lat)
    
    bottom = np.sqrt(
        sp_avg(ds_anomaly_OURS**2, w_lat) * sp_avg(ds_anomaly_ERA5**2, w_lat))
                
    acc_results.append((top/bottom).drop_vars('time'))
    
# Combine ACC results
ds_acc = xr.concat(acc_results, dim='days')

# Save
# print('Save to {}'.format(path_verif))
# # ds_acc.to_netcdf(path_verif)

Total number of initialization times: 2192


In [6]:
ds_acc['Z500']

<xarray.DataArray 'Z500' (days: 3, time: 40)>
array([[0.9990089 , 0.99844125, 0.99796839, 0.99640262, 0.99527437,
        0.99374236, 0.99263187, 0.98983759, 0.98795463, 0.98507204,
        0.98163136, 0.97733474, 0.97240235, 0.96378204, 0.95555697,
        0.94623344, 0.93836796, 0.92924667, 0.91754835, 0.90445942,
        0.88594954, 0.86632359, 0.84475436, 0.82690171, 0.80183539,
        0.7761899 , 0.74669642, 0.72041302, 0.67623802, 0.6289304 ,
        0.5757844 , 0.52577967, 0.45840988, 0.3784722 , 0.31038843,
        0.25572279, 0.21126366, 0.18649899, 0.17454886, 0.17837159],
       [0.99914724, 0.99851182, 0.99784684, 0.99701845, 0.99627861,
        0.99408991, 0.99232617, 0.9903421 , 0.98793303, 0.985086  ,
        0.98152345, 0.97576903, 0.97024137, 0.96296066, 0.9551554 ,
        0.94591987, 0.93351047, 0.91886435, 0.90165752, 0.88330607,
        0.86399491, 0.84715967, 0.82661057, 0.80775506, 0.78403124,
        0.76226468, 0.72960227, 0.69856619, 0.65923535, 0.62653755,
        0.57997605, 0.51959192, 0.46069365, 0.40906052, 0.36394108,
        0.32741154, 0.30299305, 0.28813302, 0.26377165, 0.24634158],
       [0.99905746, 0.99871384, 0.99836339, 0.99691927, 0.99603672,
        0.99473587, 0.99363908, 0.99194482, 0.99020118, 0.98625383,
        0.98260395, 0.97773928, 0.97295285, 0.96733325, 0.95927728,
        0.9486487 , 0.93577029, 0.92162886, 0.90870325, 0.89790074,
        0.88576111, 0.87337151, 0.85915401, 0.84461624, 0.82263943,
        0.80033968, 0.77377081, 0.7502817 , 0.71577929, 0.66319992,
        0.60595196, 0.5494143 , 0.49356829, 0.44136897, 0.40240682,
        0.3795396 , 0.35391542, 0.33331147, 0.31544341, 0.30963271]])
Coordinates:
    dayofyear  (days, time) int64 1 1 1 2 2 2 2 3 3 ... 10 10 10 11 11 11 11 12
    hour       (days, time) int64 6 12 18 0 6 12 18 0 6 ... 6 12 18 0 6 12 18 0
Dimensions without coordinates: days, time